# Membership Inference Attacks Against ML Models

**Reference paper:** <https://arxiv.org/abs/1610.05820>
**Reference implementation:** <https://github.com/spring-epfl/mia/>

This notebook tests an ML model using a basic membership inference attack: Determine whether a given data record was part of a model's training dataset using black-box access to the model. To conduct membership inference on a target model, we adversarially train our own inference model to detect differences in the target model's predictions for trained versus untrained inputs.

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers

from sklearn.model_selection import train_test_split

from privacy import ShadowModelBundle, AttackModelBundle, prepare_attack_data

In [2]:
NUM_CLASSES = 10
WIDTH = 32
HEIGHT = 32
CHANNELS = 3
SHADOW_DATASET_SIZE = 4000
ATTACK_TEST_DATASET_SIZE = 4000

TARGET_EPOCHS = 5 # Number of epochs to train target model.
ATTACK_EPOCHS = 5 # Number of epochs to train attack models.
SHADOW_EPOCHS = 3  # Number of epochs to train attack models.

In [3]:
def get_data():
    """Prepare CIFAR10 data."""
    (X_train, y_train), (X_test, y_test) = tf.keras.datasets.cifar10.load_data()
    y_train = tf.keras.utils.to_categorical(y_train)
    y_test = tf.keras.utils.to_categorical(y_test)
    X_train = X_train.astype("float32")
    X_test = X_test.astype("float32")
    y_train = y_train.astype("float32")
    y_test = y_test.astype("float32")
    X_train /= 255
    X_test /= 255
    return (X_train, y_train), (X_test, y_test)

In [4]:
def target_model_fn():
    """The architecture of the target (victim) model.

    The attack is white-box, hence the attacker is assumed to know this architecture too."""

    model = tf.keras.models.Sequential()

    model.add(
        layers.Conv2D(
            32,
            (3, 3),
            activation="relu",
            padding="same",
            input_shape=(WIDTH, HEIGHT, CHANNELS),
        )
    )
    model.add(layers.Conv2D(32, (3, 3), activation="relu"))
    model.add(layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(layers.Dropout(0.25))

    model.add(layers.Conv2D(64, (3, 3), activation="relu", padding="same"))
    model.add(layers.Conv2D(64, (3, 3), activation="relu"))
    model.add(layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(layers.Dropout(0.25))

    model.add(layers.Flatten())

    model.add(layers.Dense(512, activation="relu"))
    model.add(layers.Dropout(0.5))

    model.add(layers.Dense(NUM_CLASSES, activation="softmax"))
    model.compile("adam", loss="categorical_crossentropy", metrics=["accuracy"])

    return model

In [5]:
def attack_model_fn():
    """Attack model that takes target model predictions and predicts membership.

    Following the original paper, this attack model is specific to the class of the input.
    AttachModelBundle creates multiple instances of this model for each class.
    """
    model = tf.keras.models.Sequential()

    model.add(layers.Dense(128, activation="relu", input_shape=(NUM_CLASSES,)))

    model.add(layers.Dropout(0.3, noise_shape=None, seed=None))
    model.add(layers.Dense(64, activation="relu"))
    model.add(layers.Dropout(0.2, noise_shape=None, seed=None))
    model.add(layers.Dense(64, activation="relu"))

    model.add(layers.Dense(1, activation="sigmoid"))
    model.compile("adam", loss="binary_crossentropy", metrics=["accuracy"])
    return model

In [6]:
(X_train, y_train), (X_test, y_test) = get_data()

In [7]:
target_model = target_model_fn()
target_model.fit(
    X_train, y_train, epochs=TARGET_EPOCHS, validation_split=0.1, verbose=True
)

/Users/asabuncuoglu/Documents/ai-safety-demos/.venv/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/5
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 34s 24ms/step - accuracy: 0.3369 - loss: 1.7865 - val_accuracy: 0.5676 - val_loss: 1.2083
Epoch 2/5
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 34s 24ms/step - accuracy: 0.5627 - loss: 1.2104 - val_accuracy: 0.6500 - val_loss: 0.9614
Epoch 3/5
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 35s 25ms/step - accuracy: 0.6362 - loss: 1.0378 - val_accuracy: 0.7054 - val_loss: 0.8484
Epoch 4/5
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 35s 25ms/step - accuracy: 0.6719 - loss: 0.9310 - val_accuracy: 0.7252 - val_loss: 0.7886
Epoch 5/5
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 34s 24ms/step - accuracy: 0.6961 - loss: 0.8627 - val_accuracy: 0.7454 - val_loss: 0.7359


In [8]:
smb = ShadowModelBundle(
    target_model_fn,
    shadow_dataset_size=SHADOW_DATASET_SIZE,
    num_models= SHADOW_EPOCHS, # "Number of epochs to train attack models."
)

In [9]:
attacker_X_train, attacker_X_test, attacker_y_train, attacker_y_test = train_test_split(
    X_test, y_test, test_size=0.1
)
print(attacker_X_train.shape, attacker_X_test.shape)

(9000, 32, 32, 3) (1000, 32, 32, 3)


In [10]:
print("Training the shadow models...")
X_shadow, y_shadow = smb.fit_transform(
    attacker_X_train,
    attacker_y_train,
    fit_kwargs=dict(
        epochs=TARGET_EPOCHS,
        verbose=True,
        validation_data=(attacker_X_test, attacker_y_test),
    ),
)

Training the shadow models...
Epoch 1/5
125/125 ━━━━━━━━━━━━━━━━━━━━ 4s 26ms/step - accuracy: 0.1474 - loss: 2.2219 - val_accuracy: 0.3030 - val_loss: 1.8877
Epoch 2/5
125/125 ━━━━━━━━━━━━━━━━━━━━ 3s 25ms/step - accuracy: 0.3119 - loss: 1.8553 - val_accuracy: 0.3560 - val_loss: 1.6746
Epoch 3/5
125/125 ━━━━━━━━━━━━━━━━━━━━ 3s 25ms/step - accuracy: 0.3763 - loss: 1.6743 - val_accuracy: 0.3980 - val_loss: 1.5871
Epoch 4/5
125/125 ━━━━━━━━━━━━━━━━━━━━ 3s 24ms/step - accuracy: 0.4429 - loss: 1.5154 - val_accuracy: 0.4370 - val_loss: 1.4965
Epoch 5/5
125/125 ━━━━━━━━━━━━━━━━━━━━ 3s 24ms/step - accuracy: 0.4750 - loss: 1.4548 - val_accuracy: 0.4830 - val_loss: 1.4676
Epoch 1/5
125/125 ━━━━━━━━━━━━━━━━━━━━ 4s 25ms/step - accuracy: 0.1465 - loss: 2.2282 - val_accuracy: 0.2140 - val_loss: 2.0719
Epoch 2/5
125/125 ━━━━━━━━━━━━━━━━━━━━ 3s 26ms/step - accuracy: 0.2659 - loss: 2.0048 - val_accuracy: 0.3490 - val_loss: 1.7339
Epoch 3/5
125/125 ━━━━━━━━━━━━━━━━━━━━ 3s 25ms/step - accuracy: 0.3648 - l

In [11]:
# ShadowModelBundle returns data in the format suitable for the AttackModelBundle.
amb = AttackModelBundle(attack_model_fn, num_classes=NUM_CLASSES)

In [12]:
# Fit the attack models.
print("Training the attack models...")
amb.fit(
    X_shadow, y_shadow, fit_kwargs=dict(epochs=ATTACK_EPOCHS, verbose=True)
)

Training the attack models...
Epoch 1/5


/Users/asabuncuoglu/Documents/ai-safety-demos/.venv/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


75/75 ━━━━━━━━━━━━━━━━━━━━ 1s 919us/step - accuracy: 0.4897 - loss: 0.6940
Epoch 2/5
75/75 ━━━━━━━━━━━━━━━━━━━━ 0s 771us/step - accuracy: 0.5274 - loss: 0.6904
Epoch 3/5
75/75 ━━━━━━━━━━━━━━━━━━━━ 0s 727us/step - accuracy: 0.5267 - loss: 0.6902
Epoch 4/5
75/75 ━━━━━━━━━━━━━━━━━━━━ 0s 723us/step - accuracy: 0.5425 - loss: 0.6890
Epoch 5/5
75/75 ━━━━━━━━━━━━━━━━━━━━ 0s 737us/step - accuracy: 0.5324 - loss: 0.6890
Epoch 1/5
77/77 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.5054 - loss: 0.6935  
Epoch 2/5
77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 743us/step - accuracy: 0.5368 - loss: 0.6905
Epoch 3/5
77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 733us/step - accuracy: 0.5535 - loss: 0.6884
Epoch 4/5
77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 773us/step - accuracy: 0.5360 - loss: 0.6901
Epoch 5/5
77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 748us/step - accuracy: 0.5431 - loss: 0.6882
Epoch 1/5
75/75 ━━━━━━━━━━━━━━━━━━━━ 1s 944us/step - accuracy: 0.5056 - loss: 0.6925
Epoch 2/5
75/75 ━━━━━━━━━━━━━━━━━━━━ 0s 767us/step - accuracy: 0.5248 - los

In [13]:
# Test the success of the attack.

# Prepare examples that were in the training, and out of the training.
data_in = X_train[:ATTACK_TEST_DATASET_SIZE], y_train[:ATTACK_TEST_DATASET_SIZE]
data_out = X_test[:ATTACK_TEST_DATASET_SIZE], y_test[:ATTACK_TEST_DATASET_SIZE]

# Compile them into the expected format for the AttackModelBundle.
attack_test_data, real_membership_labels = prepare_attack_data(
    target_model, data_in, data_out
)

# Compute the attack accuracy.
attack_guesses = amb.predict(attack_test_data)
attack_accuracy = np.mean(attack_guesses == real_membership_labels)

print(attack_accuracy)


125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
0.52225
